# Amazon recommendation sytems


## Import libraries

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

### Reading the dataset

In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [3]:
import json
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

f = open("output.strict", 'w')
for l in parse("reviews_Home_and_Kitchen_5.json.gz"):
  f.write(l + '\n')

In [4]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Home_and_Kitchen_5.json.gz')

In [5]:
df.shape

(551682, 9)

In [6]:
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,APYOBQE6M18AA,0615391206,Martin Schwartz,"[0, 0]",My daughter wanted this book and the price on ...,5.0,Best Price,1382140800,"10 19, 2013"
1,A1JVQTAGHYOL7F,0615391206,Michelle Dinh,"[0, 0]",I bought this zoku quick pop for my daughterr ...,5.0,zoku,1403049600,"06 18, 2014"
2,A3UPYGJKZ0XTU4,0615391206,mirasreviews,"[26, 27]",There is no shortage of pop recipes available ...,4.0,"Excels at Sweet Dessert Pops, but Falls Short ...",1367712000,"05 5, 2013"
3,A2MHCTX43MIMDZ,0615391206,"M. Johnson ""Tea Lover""","[14, 18]",This book is a must have if you get a Zoku (wh...,5.0,Creative Combos,1312416000,"08 4, 2011"
4,AHAI85T5C2DH3,0615391206,PugLover,"[0, 0]",This cookbook is great. I have really enjoyed...,4.0,A must own if you own the Zoku maker...,1402099200,"06 7, 2014"


In [7]:
df.describe()

,overall,unixReviewTime
count,551682.000000,5.516820e+05
mean,4.316655,1.348687e+09
std,1.110749,6.120238e+07
min,1.000000,9.572256e+08
25%,4.000000,1.331770e+09
50%,5.000000,1.367626e+09
75%,5.000000,1.388880e+09
max,5.000000,1.406074e+09


### Recommending Based on Count

In [28]:
rating_count = pd.DataFrame(df.groupby('asin')['reviewerID'].count())
rating_AVG = pd.DataFrame(df.groupby('asin')['overall'].mean())
rating_value = pd.DataFrame(np.log(rating_count.values)*rating_AVG.values,index=rating_count.index , columns = ['Value'])
rating_value['Count'] = rating_count['reviewerID']
rating_value['AVG'] = rating_AVG['overall']
rating_value.sort_values('Value', ascending = False).head()
                                                

,Value,Count,AVG
asin,,,
B00006JSUA,32.510405,1064,4.664474
B001N07KUE,32.348679,921,4.739414
B00005B8K5,29.919394,656,4.612805
B0000CFMZN,29.170520,572,4.594406
B000M8YMEU,28.945316,550,4.587273


In [10]:
#rating_AVG.head()
rating_count.head()


,reviewerID
asin,
0615391206,11
0689027818,5
0912696591,93
1223070743,8
1567120709,16


In [11]:
rating_value.head()


,Value,Count,AVG
asin,,,
0615391206,10.681533,11,4.454545
0689027818,8.047190,5,5.000000
0912696591,20.616017,93,4.548387
1223070743,8.837627,8,4.250000
1567120709,11.263642,16,4.062500


In [12]:
product_rating_count = pd.DataFrame(df.groupby('asin')['reviewerID'].count())
user_rating_count = pd.DataFrame(df.groupby('reviewerID')['asin'].count())

product_rating_count.columns = ['countP']
user_rating_count.columns = ['countU']

filltered_product = product_rating_count[product_rating_count['countP'] > 20]
filltered_user = user_rating_count[user_rating_count['countU'] > 14]

filltered_df = df[df['reviewerID'].isin(filltered_user.index) & df['asin'].isin(filltered_product.index)]
 


In [13]:
filltered_df.shape
##product_rating_count.shape


(84755, 9)

In [15]:

review_user_table = pd.pivot_table(filltered_df, columns='asin', index='reviewerID', values='overall', fill_value=0)

In [16]:
review_user_table.head()

asin,0912696591,B00000JGRP,B00000JGRQ,B00000JGRT,B00002N5FO,B00002N5Z9,B00002N5ZB,B00002N601,B00002N602,B00002N6SQ,...,B00KM9W2PU,B00KMXNJPI,B00KO8ZDXM,B00KQJ42CW,B00KQXDIG4,B00KSRQN0Q,B00KSRSZC0,B00KSRV3UQ,B00KSRXUC0,B00KW1ZDCM
reviewerID,,,,,,,,,,,,,,,,,,,,,
A037357115LCU92V2R0YR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100L918633LUO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100UD67AHFODS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100WO06OQR8BQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1027WM3PXKVMR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Transposing the Matrix

In [17]:
review_user_table.shape

(5307, 6537)

In [18]:
X = review_user_table.T
X.shape

(6537, 5307)

### Decomposing the Matrix

In [19]:
SVD = TruncatedSVD(n_components=12, random_state=17)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(6537, 12)

### Generating a Correlation Matrix

In [20]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(6537, 6537)

In [21]:
product_id =  review_user_table.columns
product_list = list(product_id)

top_rated_product = product_list.index('B001GAQKKW')
top_rated_product

3173

In [22]:
corr_top_rated_product = corr_mat[top_rated_product]
corr_top_rated_product.shape

(6537,)

### Recommending a Highly Correlated Products (collaborative recommender system)

In [25]:
list(product_id[(corr_top_rated_product<1.0) & (corr_top_rated_product > 0.95)])

['B0001WJ5TE', 'B000VVXT5Q', 'B001GAQKKW']

In [29]:
list(product_id[(corr_top_rated_product<0.95) & (corr_top_rated_product > 0.90)])

['B00004W4UP',
 'B0000CDVD7',
 'B0000CF5JJ',
 'B0000X0X18',
 'B0001UJVTK',
 'B0002MPTUM',
 'B0007VO0DA',
 'B000A7B4CQ',
 'B000BBS94C',
 'B000F7FXF0',
 'B000X9EQ7Q',
 'B000XB45DO',
 'B001BB2LMM',
 'B001PKUXN8',
 'B001ULC93O',
 'B002U6PG06',
 'B0049U49PY',
 'B005MMNB6G',
 'B007QT4GMQ']

In [30]:
my_product = review_user_table['B00006JSUA']
filleterd_my_product = my_product[my_product>0]
filleterd_my_product.head()


reviewerID
A037357115LCU92V2R0YR    5
A1047EDJ84IMAS           5
A10BGNHGWPN7ZE           5
A10YQSHBFPFTFL           5
A117FYENI6RSPP           5
Name: B00006JSUA, dtype: int64

## Evaluating Similarity Based on Correlation## 

In [31]:
similar_to_my_product = review_user_table.corrwith(my_product)

corr_product = pd.DataFrame(similar_to_my_product, columns=['PearsonR'])
corr_product.dropna(inplace=True)
corr_product.head()

,PearsonR
asin,
0912696591,-0.010285
B00000JGRP,0.051389
B00000JGRQ,-0.006071
B00000JGRT,0.020953
B00002N5FO,0.019303
